In [10]:
from util.fft import *
from util.blurs import *
from util.images import *
from util.borders import *

from scipy.signal import convolve2d

def clip(image):
    image[image > 1] = 1
    image[image < -1] = -1

    return image

def naive_deblur_fft(B, P, center):
    # compute eigenvalues
    S = fft2(circshift(P, (0 - center[0], 0 - center[1])))
    # fix for eigenvalues equal to zero
    # S = np.where(S != math.inf, S, np.finfo(np.float64).eps)
    # compute deblurred image
    X = np.real(ifft2(np.divide(fft2(B), S)))
    return X

def blur_fft(X, P, center):
    # compute eigenvalues
    S = fft2(circshift(P, (0 - center[0], 0 - center[1])))
    # compute blurred image
    B = np.real(ifft2(np.multiply(S, fft2(X))))
    # normalize to values in range [0, 1]
    # B = (B - np.min(B))/(np.max(B) - np.min(B)) # normalizes
    return B

def pad_psf(mat: np.ndarray, size: tuple) -> np.ndarray:
    '''
    Pads a point spread funciton with zeros. The original PSF will
    appear in the upper-lerft corner in order to preserve center
    of PSF for future computations.

    Parameters:
        mat (ndarray): matrix to pad
        size (tuple): desired dimensions of the padded matrix

    Returns:
        Matrix containing the padded data.
    '''

    zeros = np.zeros(size)
    zeros[0:mat.shape[0], 0:mat.shape[1]] = mat
    return zeros

psf_fn     = psf_gaussian
psf_size   = (6, 6)
psf_center = tuple(int(i/2) for i in psf_size)
psf_params = [psf_size, 2, 2, psf_center[0], psf_center[1]]

noise      = 0.1
image_name = 'pumpkins'
image_path = 'samples/{}.png'.format(image_name)

# load true image from path, values are normalized to [0, 1]
true_image = load_img(image_path)

# apply boundary conditions to true image
# boundary_image = periodic(true_image, tuple(i*2 for i in psf_size))
# save_img_raw(boundary_image, 'results/1_{}_boundary.png'.format(image_name))

# create PSF matrix
psf = psf_fn(*psf_params)
save_img_raw(psf, 'results/2_{}_psf.png'.format(image_name))

# convolve the true image with the PSF to blur it
psf_padded = pad_psf(psf, true_image.shape)
blurred_image = blur_fft(true_image, psf_padded, psf_center) + 1e-12
save_img_raw(blurred_image, 'results/3_{}_blurred.png'.format(image_name))

deblurred_image = naive_deblur_fft(psf_padded, blurred_image, psf_center)
save_img_raw(deblurred_image, 'results/4_{}_deblurred.png'.format(image_name))

# add noise to the blurred image
# noisy_image = add_noise(blurred_image, noise)
# save_img_raw(noisy_image, 'results/4_{}_noisy.png'.format(image_name))

# for i in range(10):
#     # create PSF from minimized s
#     minimized_psf = 